# Intersection Volume

In [1]:
# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

### Volume for an Edge-Vertex contact.
We compute contact volumes per-edge using the edges earliest contact. For each edge, the volume is given by
$$
\newcommand{\bU}{\mathbf{U}}
\newcommand{\be}{\mathbf{e}}
V_k(\be) = (1 - \tau_I)\sqrt{\epsilon^2 |\be(\tau_I)|^2 + \bU_{ij} \cdot \be(\tau_I)^{\bot}}
$$

where 
- $|e(\tau_I)|$ is the length of the edge at the first time of collision ($\tau_I$), 
- $\epsilon$ is some small value
- $\bU_{ij}$ is the velocity of the contact point at the edge. 

Let $\bU_i,\bU_j,\bU_k$ be the velocities (i.e also distances since $dt=1$) of the three vertexes. We can compute $\bU_{ij}$, the velocity of the contact point along the edge as

$$ \bU_{ij} = \bU_{i} + \alpha_{\tau} (\bU_{j} - \bU_{i}) $$

where $\alpha_{\tau}$ is the distance along the edge where the first contact happens ( $0\leq \alpha \leq 1$).


In [2]:
import sympy as sp
import numpy as np
from sympy import init_printing
init_printing()

In [4]:
v = np.array(sp.symbols("V_i:k")) # vertices positions
d = np.array(sp.symbols("U_i:k")) # vertices displacements
alpha = sp.symbols("\\alpha")
t = sp.symbols("t")
tau = sp.symbols("\\tau_I")

In [8]:
v_tau = v + tau * d
v_tau

array([U_i*\tau_I + V_i, U_j*\tau_I + V_j, U_k*\tau_I + V_k], dtype=object)

In [7]:
e = v_tau[1] - v_tau[0]
e